### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34767 entries, 0 to 34766
Data columns (total 3 columns):
movie_id    34767 non-null object
movie       34767 non-null object
genre       34501 non-null object
dtypes: object(3)
memory usage: 814.9+ KB


In [3]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [16]:
movies.genre.value_counts()[:10]

Drama                   3503
Comedy                  2031
Documentary             1396
Comedy|Drama            1335
Drama|Romance           1170
Comedy|Drama|Romance     979
Comedy|Romance           839
Horror                   805
Horror|Thriller          553
Drama|Thriller           496
Name: genre, dtype: int64

In [17]:
len(movies.genre.value_counts())

2705

In [46]:
movies.genre.dropna().head()

0      Documentary|Short
1      Documentary|Short
2      Documentary|Short
4           Short|Horror
5    Short|Comedy|Horror
Name: genre, dtype: object

In [47]:
all_genres = []
for x in movies.genre.dropna():
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres[:10]

array(['Documentary', 'Short', 'Horror', 'Comedy', 'Action', 'Adventure',
       'Fantasy', 'Sci-Fi', 'Crime', 'Western'], dtype=object)

In [48]:
len(genres)

28

In [49]:
genres

array(['Documentary', 'Short', 'Horror', 'Comedy', 'Action', 'Adventure',
       'Fantasy', 'Sci-Fi', 'Crime', 'Western', 'Drama', 'Romance',
       'History', 'Family', 'War', 'Sport', 'Biography', 'Mystery',
       'Thriller', 'Animation', 'Music', 'Musical', 'Film-Noir', 'Adult',
       'Talk-Show', 'News', 'Reality-TV', 'Game-Show'], dtype=object)

In [6]:
# more work
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491127 entries, 0 to 491126
Data columns (total 4 columns):
user_id      491127 non-null object
movie_id     491127 non-null object
rating       491126 non-null float64
timestamp    491126 non-null object
dtypes: float64(1), object(3)
memory usage: 15.0+ MB


In [7]:
# and still more
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0111161,10.0,1373234211
1,1,0117060,7.0,1373415231
2,1,0120755,6.0,1373424360
3,1,0317919,6.0,1373495763
4,1,0454876,10.0,1373621125


In [10]:
# and even more - open additional cells as necessary
len(np.unique(reviews.user_id))

36178

In [11]:
reviews.describe()

,rating
count,491126.000000
mean,7.300178
std,1.873090
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [50]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 34767,
'The number of ratings in the dataset': 491127,
'The number of different genres': 28,
'The number of unique users in the dataset': 36178,
'The number missing ratings in the reviews dataset': 1,
'The average rating given across all ratings': 7.300178,
'The minimum rating given across all ratings': 0.000000,
'The maximum rating given across all ratings': 10.000000,
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea
dict_sol1

{'The number of movies in the dataset': 34767,
 'The number of ratings in the dataset': 491127,
 'The number of different genres': 28,
 'The number of unique users in the dataset': 36178,
 'The number missing ratings in the reviews dataset': 1,
 'The average rating given across all ratings': 7.300178,
 'The minimum rating given across all ratings': 0.0,
 'The maximum rating given across all ratings': 10.0}

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [88]:
pattern = " \(([0-9]{4})\)$"
pattern

' \\(([0-9]{4})\\)$'

In [89]:
# cell for work
movies.movie.head()

0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumière (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [90]:
matches = movies.movie.str.findall(pattern)
matches.head()

0    [1894]
1    [1895]
2    [1896]
3    [1895]
4    [1896]
Name: movie, dtype: object

In [91]:
len(matches)

34767

In [92]:
# and another
movies["year"] = matches.str[0]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34767 entries, 0 to 34766
Data columns (total 4 columns):
movie_id    34767 non-null object
movie       34767 non-null object
genre       34501 non-null object
year        34767 non-null object
dtypes: object(4)
memory usage: 1.1+ MB


In [93]:
movies.head()

,movie_id,movie,genre,year
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [111]:
# # and a few more below, which you can use as necessary
# zero_matrix = np.zeros((len(movies), len(genres)))
# dummies = pd.DataFrame(zero_matrix, columns=genres)
# dummies.info()

In [112]:
# dummies.describe()

In [110]:
# for i, gen in enumerate(genres):
#     indices = dummies.columns.get_indexer(gen.split('|'))
#     dummies.iloc[i, indices] = 1

# dummies.describe()

In [108]:
# movies_windic = movies.join(dummies.add_prefix('Genre_'))
# movies_windic.iloc[0]

In [109]:
# movies_windic.iloc[10000]

In [113]:
# Function to split and return values for columns
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)
    
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34767 entries, 0 to 34766
Data columns (total 32 columns):
movie_id       34767 non-null object
movie          34767 non-null object
genre          34501 non-null object
year           34767 non-null object
Documentary    34767 non-null int64
Short          34767 non-null int64
Horror         34767 non-null int64
Comedy         34767 non-null int64
Action         34767 non-null int64
Adventure      34767 non-null int64
Fantasy        34767 non-null int64
Sci-Fi         34767 non-null int64
Crime          34767 non-null int64
Western        34767 non-null int64
Drama          34767 non-null int64
Romance        34767 non-null int64
History        34767 non-null int64
Family         34767 non-null int64
War            34767 non-null int64
Sport          34767 non-null int64
Biography      34767 non-null int64
Mystery        34767 non-null int64
Thriller       34767 non-null int64
Animation      34767 non-null int64
Music          34767 no

In [114]:
# Return century of movie as a dummy column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['year'].apply(add_movie_year)

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34767 entries, 0 to 34766
Data columns (total 35 columns):
movie_id       34767 non-null object
movie          34767 non-null object
genre          34501 non-null object
year           34767 non-null object
Documentary    34767 non-null int64
Short          34767 non-null int64
Horror         34767 non-null int64
Comedy         34767 non-null int64
Action         34767 non-null int64
Adventure      34767 non-null int64
Fantasy        34767 non-null int64
Sci-Fi         34767 non-null int64
Crime          34767 non-null int64
Western        34767 non-null int64
Drama          34767 non-null int64
Romance        34767 non-null int64
History        34767 non-null int64
Family         34767 non-null int64
War            34767 non-null int64
Sport          34767 non-null int64
Biography      34767 non-null int64
Mystery        34767 non-null int64
Thriller       34767 non-null int64
Animation      34767 non-null int64
Music          34767 no

In [115]:
movies.describe()

,Documentary,Short,Horror,Comedy,Action,Adventure,Fantasy,Sci-Fi,Crime,Western,...,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show,1800's,1900's,2000's
count,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,...,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000,34767.000000
mean,0.093738,0.040297,0.124486,0.311876,0.150430,0.090459,0.063221,0.068686,0.137084,0.014353,...,0.023844,0.007967,0.000288,0.000086,0.003193,0.000230,0.000029,0.000316,0.317801,0.681882
std,0.291469,0.196657,0.330140,0.463266,0.357498,0.286843,0.243363,0.252923,0.343941,0.118942,...,0.152567,0.088905,0.016957,0.009289,0.056414,0.015168,0.005363,0.017785,0.465628,0.465752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [120]:
movies.head()

,movie_id,movie,genre,year,Documentary,Short,Horror,Comedy,Action,Adventure,...,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [117]:
reviews['date'] = pd.to_datetime(reviews['timestamp'],unit='s')
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491127 entries, 0 to 491126
Data columns (total 5 columns):
user_id      491127 non-null object
movie_id     491127 non-null object
rating       491126 non-null float64
timestamp    491126 non-null object
date         491126 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 18.7+ MB


In [118]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0111161,10.0,1373234211,2013-07-07 21:56:51
1,1,0117060,7.0,1373415231,2013-07-10 00:13:51
2,1,0120755,6.0,1373424360,2013-07-10 02:46:00
3,1,0317919,6.0,1373495763,2013-07-10 22:36:03
4,1,0454876,10.0,1373621125,2013-07-12 09:25:25


In [119]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year